In [268]:
from __future__ import division
%matplotlib tk

import os, sys, time
import h5py as hp
import numpy as np
import matplotlib.pyplot as plt

# Added by Themis
import scipy
sys.path.append('/Applications/anaconda/pkgs/')

from scipy.constants import m_p, c, e
from mpl_toolkits.mplot3d import Axes3D

from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.particles.generators import Gaussian6D, MatchRFBucket6D, ImportDistribution
from PyHEADTAIL.trackers.detuners import Chromaticity, AmplitudeDetuning
from PyHEADTAIL.trackers.transverse_tracking import TransverseMap
from PyHEADTAIL.trackers.simple_long_tracking import RFSystems, LinearMap
from PyHEADTAIL.monitors.monitors import BunchMonitor, SliceMonitor, ParticleMonitor
from PyHEADTAIL.feedback.transverse_damper import TransverseDamper
from PyHEADTAIL.impedances.wakes import CircularResonator, WakeTable, WakeField

In [269]:
plt.close('all')

In [270]:
n_turns = 100000
decTurns = 100 # how often to record data

gamma          = 7460.5232328
p0             = m_p*c*np.sqrt(gamma**2 - 1)
beta           = np.sqrt(1 - 1/gamma**2)
circumference  = 26658.883

In [271]:
# CREATE TRANSVERSE MAP
# =====================
n_segments     = 1
s              = np.arange(0, n_segments + 1) * circumference / n_segments
alpha_x        = 0 * np.ones(n_segments)
beta_x         = 65.9756337546 * np.ones(n_segments)
D_x            = 0 * np.ones(n_segments)
alpha_y        = 0 * np.ones(n_segments)
beta_y         = 71.5255058456 * np.ones(n_segments)
D_y            = 0 * np.ones(n_segments)

Q_x            = 62.31
Q_y            = 60.32

Qp_x           = 10
Qp_y           = 10

app_x          = 3e-8
app_y          = 3e-8
app_xy         = 0 #-2.25e-8

transverse_map = TransverseMap(
    circumference, s, alpha_x, beta_x, D_x, alpha_y, beta_y, D_y, Q_x, Q_y,
    Chromaticity(Qp_x, Qp_y),
    AmplitudeDetuning(app_x, app_y, app_xy))

In [272]:
# CREATE LONGITUDINAL MAP
# =======================
alpha           = 0.0003225
Q_s             = 0.002044761458
h1, h2          = 35640, 71280
V1, V2          = 16e6, 8e6
dphi1, dphi2    = 0, np.pi
p_increment     = 0 * e/c * circumference/(beta*c)

longitudinal_map = LinearMap([alpha], circumference, Q_s)

In [273]:
# CREATE DAMPER
# =============
dampingrate_x = 20
dampingrate_y = 20
damper = TransverseDamper(dampingrate_x, dampingrate_y)

In [274]:
# CREATE BEAM
# ===========
macroparticlenumber = 100000

charge    = e
mass      = m_p
intensity = 1.5e11

R         = circumference/(2*np.pi)
eta       = alpha-1/gamma**2
beta_z    = np.abs(eta)*R/Q_s
epsn_x    = 2.5e-6
epsn_y    = 2.5e-6
epsn_z    = 2.5
sigma_z   = 0.075

sigma_x   = np.sqrt(epsn_x/(beta*gamma) * beta_x[0])
sigma_xp  = sigma_x/beta_x[0]
sigma_y   = np.sqrt(epsn_y/(beta*gamma) * beta_y[0])
sigma_yp  = sigma_y/beta_y[0]
sigma_dp  = sigma_z/beta_z
epsn_z    = 4*np.pi * p0/e * sigma_z*sigma_dp

bunch     = Gaussian6D(
    macroparticlenumber, intensity, charge, mass, circumference, gamma,
    sigma_x, sigma_xp, sigma_y, sigma_yp, sigma_z, sigma_dp
    ).generate()

xoffset = 0e-4
yoffset = 0e-4
bunch.x += xoffset
bunch.y += yoffset

import pickle
afile = open('./../../../Desktop/data/bunch1', 'wb')
pickle.dump(bunch, afile)
afile.close()

In [275]:
plt.close('all')

In [276]:
# ======================================================================
# SET UP ACCELERATOR MAP AND START TRACKING
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

import csv

for k in range(10):
#     plt.ion()
#     fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3,3, figsize=(16,9))

    import pickle

    #reload object from file
    file2 = open('./../../../Desktop/data/bunch1', 'rb')
    bunch = pickle.load(file2)
    file2.close()

    print '--> Begin tracking...'
#     one_turn_map = [transverse_map[0]] + [longitudinal_map] #+ [damper] #+ [wakes]
    z0 = bunch.z

    meanX = np.zeros(n_turns/decTurns)
    meanY = np.zeros(n_turns/decTurns)
    meanXsq = np.zeros(n_turns/decTurns)
    meanYsq = np.zeros(n_turns/decTurns)
    emitX = np.zeros(n_turns/decTurns)
    emitY = np.zeros(n_turns/decTurns)

#     import scipy.io as sio
#     mat=sio.loadmat('./../../../Desktop/data/noiseFiles_%s.mat'%k)
#     noiseX = mat['noiseX'] # sigma of 8
#     noiseY = mat['noiseY'] # sigma of 8

    for i in range(n_turns):
        t0 = time.clock()
        for m in one_turn_map:
            m.track(bunch)

        # INJECT NOISE HERE AT EVERY TURN ON BEAM
        # ======================================================

        # Shaped Phase noise
#         bunch.xp += 1.75e-11*noiseX[i]*np.cos(2*np.pi*400e6/(bunch.beta*c)*bunch.z)
#         bunch.yp += 2e-11*noiseY[i]*np.cos(2*np.pi*400e6/(bunch.beta*c)*bunch.z)

        # Shaped Amplitude noise
#         bunch.xp += 2.66e-11*noiseX[i]*np.sin(2*np.pi*400e6/(bunch.beta*c)*bunch.z)
#         bunch.yp += 3e-11*noiseY[i]*np.sin(2*np.pi*400e6/(bunch.beta*c)*bunch.z)

        # Gaussian Phase noise
        bunch.xp += np.random.normal(0, 2e-9)*np.cos(2*np.pi*400e6/(bunch.beta*c)*bunch.z)
#         bunch.xp -= 2/dampingrate_x*np.mean(bunch.xp)
#         bunch.yp += np.random.normal(0, 2.25e-9)*np.cos(2*np.pi*400e6/(bunch.beta*c)*bunch.z)

        # Gaussian Amplitude noise
#         bunch.xp += np.random.normal(0, 6e-9)*np.sin(2*np.pi*400e6/(bunch.beta*c)*bunch.z)
#         bunch.yp += np.random.normal(0, 3.5e-9)*np.sin(2*np.pi*400e6/(bunch.beta*c)*bunch.z)
        # ======================================================

        if i%decTurns is  0:
            meanX[i/decTurns] = np.mean(bunch.x)
            meanY[i/decTurns] = np.mean(bunch.y)
            meanXsq[i/decTurns] = np.mean(bunch.x**2)
            meanYsq[i/decTurns] = np.mean(bunch.y**2)
            emitX[i/decTurns] = bunch.epsn_x()
            emitY[i/decTurns] = bunch.epsn_y()

#             print 'turn #{:d}'.format(i)
#             Jx = bunch.x**2 + (beta_x * bunch.xp)**2
#             Jy = bunch.y**2 + (beta_y * bunch.yp)**2
#             Jz = bunch.z**2 + (beta_z * bunch.dp)**2
#             ax1.scatter(bunch.x, bunch.xp, c=Jz, lw=0.1)
#             ax2.scatter(bunch.y, bunch.yp, c=Jz, lw=0.1)
#             ax3.scatter(bunch.z, bunch.dp, c=Jz, lw=0.1)
#             ax4.plot(i, np.mean(bunch.x), 'rx')
#             ax4.plot(i, xoffset*np.exp(-(i+1)/50), 'k*')
#             ax5.plot(i, np.mean(bunch.y), 'rx')
#             ax5.plot(i, yoffset*np.exp(-(i+1)/50), 'k*')
#             ax6.plot(i, np.mean(bunch.z), 'rx')
#             ax7.plot(i, bunch.epsn_x()*1e6, 'gx')
#             ax8.plot(i, bunch.epsn_y()*1e6, 'gx')
#             ax9.plot(i, bunch.epsn_z(), 'gx')

#             ax1.set_xlabel('x'); ax1.set_ylabel('xp')
#             ax4.set_ylabel('Mean x position')
#             ax7.set_xlabel('Turn'); ax7.set_ylabel('$\epsilon_x [\mu m]$')

#             ax2.set_xlabel('y'); ax2.set_ylabel('yp')
#             ax5.set_ylabel('Mean y position')
#             ax8.set_xlabel('Turn'); ax8.set_ylabel('$\epsilon_y [\mu m]$')

#             ax3.set_xlabel('s'); ax3.set_ylabel('$\delta$')
#             ax6.set_ylabel('Mean z position')
#             ax9.set_xlabel('Turn'); ax9.set_ylabel('$\epsilon_z$ [eVs]')

#             [ax.set_xlim(-2e-3, 2e-3) for ax in (ax1, ax2)]
#             [ax.set_ylim(-2e-5, 2e-5) for ax in (ax1, ax2)]
#             ax3.set_xlim(-5e-1, 5e-1); ax3.set_ylim(-5e-4, 5e-4)
#             plt.tight_layout()
#             plt.draw()
#             ax1.cla(); ax2.cla(); ax3.cla()

    print k
    dataExport = [meanX, meanY, meanXsq, meanYsq, emitX, emitY]
    with open('./../../../Desktop/data/filelong_%s.txt'%k,'wb') as f:
        out = csv.writer(f, delimiter=',')
        out.writerows(zip(*dataExport))

print '--> Done.'

# plt.close('all')



--> Begin tracking...
0
--> Done.


In [277]:
# Jx = (bunch.x**2 + (beta_x * bunch.xp)**2)/(2*beta_x)
# Jy = (bunch.y**2 + (beta_y * bunch.yp)**2)/(2*beta_y)
# dq = app_x*Jx/bunch.p0+app_xy*Jy/bunch.p0+Qp_x*bunch.dp

# print np.std(dq)*11245

# dataExport = [dq]
# with open('./../../../Desktop/data/dq.txt','wb') as f:
#     out = csv.writer(f, delimiter=',')
#     out.writerows(zip(*dataExport))

# Expected damper improvement NEEDS TO BE UPDATED!

# rho, bin_edges = np.histogram((dq + .31)*11245, 100)
# deltaF =  bin_edges[2]-bin_edges[1]
# rho = rho/np.sum(rho)/deltaF;
# Hbeam = rho/4j;

# gain = 11245/(dampingrate_x); # gain = 2/damping rate. Damping rate = 10

# integrand = rho/(np.abs(1+1j*gain*Hbeam))**2;
# corrFactor = np.sum(integrand)*deltaF
# print corrFactor

30.3678527296


In [278]:
# dataExport = [bunch.xp]
# with open('./../../../Desktop/data/initialXphighNoise.txt','wb') as f:
#     out = csv.writer(f, delimiter=',')
#     out.writerows(zip(*dataExport))

In [279]:
import os
os.system('say "your program has finished"')

0